In [15]:
import json
from datetime import datetime
from collections import defaultdict

# Clase Producto
class Producto:
    def __init__(self, nombre, fecha_vencimiento=None, perecible=False, cantidad=0, categoria=None):
        self.nombre = nombre
        self.fecha_vencimiento = datetime.strptime(fecha_vencimiento, "%Y-%m-%d") if fecha_vencimiento else None
        self.perecible = perecible
        self.cantidad = cantidad
        self.categoria = categoria
        self.siguiente = None  # Enlace al siguiente producto

# Clase Nodo
class Nodo:
    def __init__(self, tipo):
        self.tipo = tipo
        self.producto = None
        self.hijo = None  # Primer subcategoría o producto
        self.hermano = None  # Siguiente nodo del mismo nivel

    def agregar_producto(self, nuevo_producto):
        if not self.producto:
            self.producto = nuevo_producto
        else:
            actual = self.producto
            anterior = None
            while actual and (not nuevo_producto.perecible or 
                              (nuevo_producto.perecible and nuevo_producto.fecha_vencimiento >= actual.fecha_vencimiento)):
                anterior = actual
                actual = actual.siguiente
            nuevo_producto.siguiente = actual
            if anterior:
                anterior.siguiente = nuevo_producto
            else:
                self.producto = nuevo_producto

    def agregar_hijo(self, nodo_hijo):
        if not self.hijo:
            self.hijo = nodo_hijo
        else:
            actual = self.hijo
            while actual.hermano:
                actual = actual.hermano
            actual.hermano = nodo_hijo

    def buscar_producto_por_tipo(self, tipo):
        if self.tipo == tipo:
            return self
        actual = self.hijo
        while actual:
            resultado = actual.buscar_producto_por_tipo(tipo)
            if resultado:
                return resultado
            actual = actual.hermano
        return None
    
    def buscar_producto_por_vencimiento(self, fecha):
        fecha_busqueda = datetime.strptime(fecha, "%Y-%m-%d")
        producto_actual = self.producto
        while producto_actual:
            if producto_actual.perecible and producto_actual.fecha_vencimiento <= fecha_busqueda:
                print(f"Producto que vence antes del {fecha}: {producto_actual.nombre}")
            producto_actual = producto_actual.siguiente
        hijo = self.hijo
        while hijo:
            hijo.buscar_producto_por_vencimiento(fecha)
            hijo = hijo.hermano

# Clase Arbol
class Arbol:
    def __init__(self, tipo_raiz):
        self.raiz = Nodo(tipo_raiz)

    def agregar_tipo(self, nodo_tipo):
        self.raiz.agregar_hijo(nodo_tipo)

    def agregar_producto_a_tipo(self, tipo, producto):
        nodo_tipo = self.raiz.buscar_producto_por_tipo(tipo)
        if nodo_tipo:
            nodo_tipo.agregar_producto(producto)
        else:
            print(f"Tipo de producto '{tipo}' no encontrado.")
    
    def buscar_por_tipo(self, tipo):
        nodo_tipo = self.raiz.buscar_producto_por_tipo(tipo)
        return nodo_tipo.producto if nodo_tipo else None

    def buscar_por_vencimiento(self, fecha):
        self.raiz.buscar_producto_por_vencimiento(fecha)

    def calcular_suma_cantidad_por_tipo(self):
        cantidades = defaultdict(int)
        self._calcular_suma_cantidad_por_tipo_recursivo(self.raiz, cantidades)
        return dict(cantidades)

    def _calcular_suma_cantidad_por_tipo_recursivo(self, nodo, cantidades):
        if nodo.producto:
            producto_actual = nodo.producto
            while producto_actual:
                cantidades[nodo.tipo] += producto_actual.cantidad
                producto_actual = producto_actual.siguiente
        if nodo.hijo:
            self._calcular_suma_cantidad_por_tipo_recursivo(nodo.hijo, cantidades)
        if nodo.hermano:
            self._calcular_suma_cantidad_por_tipo_recursivo(nodo.hermano, cantidades)

    def calcular_suma_cantidad_por_nombre(self):
        nombres = defaultdict(int)
        self._calcular_suma_cantidad_por_nombre_recursivo(self.raiz, nombres)
        return dict(nombres)

    def _calcular_suma_cantidad_por_nombre_recursivo(self, nodo, nombres):
        if nodo.producto:
            producto_actual = nodo.producto
            while producto_actual:
                nombres[producto_actual.nombre] += producto_actual.cantidad
                producto_actual = producto_actual.siguiente
        if nodo.hijo:
            self._calcular_suma_cantidad_por_nombre_recursivo(nodo.hijo, nombres)
        if nodo.hermano:
            self._calcular_suma_cantidad_por_nombre_recursivo(nodo.hermano, nombres)

    def calcular_suma_cantidad_por_categoria(self):
        categorias = defaultdict(int)
        self._calcular_suma_cantidad_por_categoria_recursivo(self.raiz, categorias)
        return dict(categorias)

    def _calcular_suma_cantidad_por_categoria_recursivo(self, nodo, categorias):
        if nodo.producto:
            producto_actual = nodo.producto
            while producto_actual:
                categorias[producto_actual.categoria] += producto_actual.cantidad
                producto_actual = producto_actual.siguiente
        if nodo.hijo:
            self._calcular_suma_cantidad_por_categoria_recursivo(nodo.hijo, categorias)
        if nodo.hermano:
            self._calcular_suma_cantidad_por_categoria_recursivo(nodo.hermano, categorias)

# Función para cargar datos desde archivo
def cargar_datos_desde_archivo(nombre_archivo, arbol):
    with open(nombre_archivo, 'r') as archivo:
        lineas = archivo.readlines()
        for linea in lineas:
            datos = json.loads(linea.strip())
            tipo = "Perecedero" if datos["tipo"] == "perecedero" else "No perecedero"
            producto = Producto(
                nombre=datos["nombre"],
                fecha_vencimiento=datos.get("fech_venc"),
                perecible=(datos["tipo"] == "perecedero"),
                cantidad=datos["cantidad"],
                categoria=datos["categoria"]
            )
            # Buscar o crear el nodo del tipo correspondiente
            nodo_tipo = arbol.raiz.buscar_producto_por_tipo(tipo)
            if not nodo_tipo:
                nodo_tipo = Nodo(tipo)
                arbol.agregar_tipo(nodo_tipo)
            # Agregar el producto al nodo
            nodo_tipo.agregar_producto(producto)

# Crear el árbol de productos
arbol_productos = Arbol("Almacén")

# Cargar datos desde inventario.txt
cargar_datos_desde_archivo("inventario1.txt", arbol_productos)


In [16]:
# Mostrar los productos perecederos y no perecederos
perecederos = arbol_productos.buscar_por_tipo("Perecedero")
no_perecederos = arbol_productos.buscar_por_tipo("No perecedero")

print("Productos perecederos:")
while perecederos:
    print(f"- {perecederos.nombre}, vence: {perecederos.fecha_vencimiento.strftime('%Y-%m-%d')}")
    perecederos = perecederos.siguiente

print("\nProductos no perecederos:")
while no_perecederos:
    print(f"- {no_perecederos.nombre}")
    no_perecederos = no_perecederos.siguiente

# Buscar productos perecederos que vencen antes de una fecha
print("\nProductos perecederos que vencen antes de 2024-11-05:")
arbol_productos.buscar_por_vencimiento("2024-11-05")

Productos perecederos:
- Manzana, vence: 2024-10-26
- Pollo, vence: 2024-10-27
- Naranja, vence: 2024-11-15
- Platano, vence: 2024-12-12
- Leche, vence: 2025-11-10

Productos no perecederos:
- Arroz
- Lentejas
- Pasta

Productos perecederos que vencen antes de 2024-11-05:
Producto que vence antes del 2024-11-05: Manzana
Producto que vence antes del 2024-11-05: Pollo


In [17]:
# Cálculo de sumas
print("Suma de cantidades por tipo:")
suma_por_tipo = arbol_productos.calcular_suma_cantidad_por_tipo()
print(", ".join([f"{tipo}: {cantidad}" for tipo, cantidad in suma_por_tipo.items()]))

print("\nSuma de cantidades por nombre:")
suma_por_nombre = arbol_productos.calcular_suma_cantidad_por_nombre()
print(", ".join([f"{nombre}: {cantidad}" for nombre, cantidad in suma_por_nombre.items()]))

print("\nSuma de cantidades por categoría:")
suma_por_categoria = arbol_productos.calcular_suma_cantidad_por_categoria()
print(", ".join([f"{categoria}: {cantidad}" for categoria, cantidad in suma_por_categoria.items()]))

Suma de cantidades por tipo:


TypeError: unsupported operand type(s) for +=: 'int' and 'str'